## Generating and Visualizing Antipodal Grasps on Book

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import trimesh
import sys
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud,
    Simulator,
    TrajectorySource,
    Solve,
    RotationMatrix,
    MultibodyPlant,
    eq,
    StateInterpolatorWithDiscreteDerivative,
    MinimumDistanceLowerBoundConstraint,
    RollPitchYaw,
    SolutionResult,
    CollisionFilterDeclaration,
    GeometrySet,
    Role,
)
from pydrake.geometry import Meshcat
from pydrake.multibody import inverse_kinematics

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb, AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
# Own utils
from hwstation.utils import * 
from hwstation.add_objects import *
from perception.icp import run_table_multi_book_icp, check_quality_of_icp
from rrt.rrt_planning import *
from grasp.grasping import *

from enum import Enum
import pandas as pd

from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.station import AddPointClouds
from random import random
from manipulation.pick import *
#from manipulation.pick import *

## 1. Setup environment

In [3]:
# Start meshcat
try:
    meshcat = Meshcat(7009)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)


INFO:drake:Meshcat listening for connections at http://localhost:7009


In [4]:
shelf_labels = ["A","B"] #,"C","D","E","F"]
shelf_pos_list = [(1,1),(0,0)]
N_BOOKS_TO_GRASP = 2
book_grasp_dur = 10.0
Q_TRAJECTORY_FPATH = "trajectories/2_q_book_traj"
WSG_TRAJECTORY_FPATH = "trajectories/2_wsg_book_traj"

In [5]:
time_point_list = [(None, 0.0)]
for idx in range(N_BOOKS_TO_GRASP):
    end_time = time_point_list[-1][1]
    book_grasp_time = end_time + book_grasp_dur
    
    shelf_placement_dur = 20.0

    if shelf_labels[idx] in ["B", "C"]:
        shelf_placement_dur = 30.0

    if shelf_labels[idx] in ["D", "E"]:
        shelf_placement_dur = 40.0

    if shelf_labels[idx] == "F":
        shelf_placement_dur = 60.0

    book_pl_time = book_grasp_time + shelf_placement_dur
    time_point_list.append((book_grasp_time,book_pl_time))

In [6]:
q_traj_final = load_traj(Q_TRAJECTORY_FPATH)
traj_wsg_final = load_traj(WSG_TRAJECTORY_FPATH)


(100, 11)
(100, 2)


In [7]:
scenario_data = get_library_scenario_data(cameras=False)
meshcat.Delete()
builder = DiagramBuilder()
scenario = load_scenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat,parser_preload_callback=ConfigureParser))


# COMMENTING OUT THE POINT CLOUD STUFF FOR NOW FOR THE PURPOSE OF VISUALIZING
# # Adding point cloud extractors:
# to_point_cloud = AddPointClouds(
#     scenario=scenario, station=station, builder=builder, meshcat=meshcat
# )

# # Connect point clouds with output port:
# for idx, name in enumerate(to_point_cloud.keys()):
#     builder.ExportOutput(
#         to_point_cloud[name].get_output_port(), name+"_ptcloud")

builder.ExportOutput(
    station.GetOutputPort("mobile_iiwa.state_estimated"),
    "mobile_iiwa.state_estimated"
)

# builder.ExportInput(
#     station.GetInputPort("mobile_iiwa.desired_state"),
#     "mobile_iiwa.desired_state"
# )

q_traj_system = builder.AddSystem(TrajectorySource(q_traj_final))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_final))

state_interpolator = builder.AddSystem(StateInterpolatorWithDiscreteDerivative(num_positions=10, time_step=0.1))

# builder.Connect(
#     q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
# )

builder.Connect(
    q_traj_system.get_output_port(), state_interpolator.GetInputPort("position")
)

builder.Connect(
    state_interpolator.GetOutputPort("state"), station.GetInputPort("mobile_iiwa.desired_state")
)

builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
)


visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()
#diagram.set_name("plant and scene_graph")
diagram_context = diagram.CreateDefaultContext()
scene_graph = diagram.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_context)
filterCollsionGeometry(scene_graph, sg_context)
simulator = Simulator(diagram)

In [8]:
visualizer.StartRecording(False)
simulator.AdvanceTo(time_point_list[-1][1])
visualizer.PublishRecording()

In [ ]:
# visualizer.StartRecording(False)
# simulator.AdvanceTo(2*time_point_list[-1][1]/4)
# visualizer.PublishRecording()

In [ ]:
# visualizer.StartRecording(False)
# simulator.AdvanceTo(3*time_point_list[-1][1]/4)
# visualizer.PublishRecording()

In [ ]:
# visualizer.StartRecording(False)
# simulator.AdvanceTo(time_point_list[-1][1])
# visualizer.PublishRecording()